In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

Loading imports


In [3]:
torch.cuda.set_device(6)

In [4]:
Meta.df_comb.head()

,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,Rows,...,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
SOPInstanceUID,,,,,,,,,,,,,,,,,,,,,
ID_76d55d9d0,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_96d282ea9,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_7d8a7c29d,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_4d4401491,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_8f5ded0b7,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


## Training

In [6]:
name = 'baseline_full_stg1_xse_resnext'

In [7]:
dbch = get_data(512, 128, splits=Meta.splits_stg1)
learn = get_learner(dbch, xse_resnext18, pretrained=True)

## Training

In [8]:
do_fit(learn, 12, 4e-2)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.123301,0.128679,0.960710,0.925240,08:57
1,0.113026,0.119885,0.964071,0.923878,08:42
2,0.104065,0.138082,0.962177,0.918225,08:55
3,0.100908,0.118655,0.966079,0.927570,09:17
4,0.095516,0.104754,0.968527,0.939143,09:07
5,0.093690,0.101034,0.970096,0.938328,08:46
6,0.090035,0.098093,0.970785,0.941027,08:49
7,0.083441,0.095108,0.971008,0.941486,09:31
8,0.080790,0.092370,0.972799,0.944771,09:09
9,0.075252,0.089204,0.973892,0.946871,09:29


In [9]:
learn.dbunch = get_data(512, 256, splits=Meta.splits_stg1)
do_fit(learn, 12, 4e-3)
learn.save(f'runs/{name}-2')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.081481,0.092056,0.972351,0.944580,20:37
1,0.076591,0.092315,0.973295,0.945764,20:09
2,0.076864,0.084091,0.973854,0.948374,20:14
3,0.071800,0.084416,0.974747,0.949532,20:11
4,0.069287,0.083371,0.974932,0.949277,19:34
5,0.066857,0.081601,0.975528,0.951646,19:13
6,0.065854,0.085698,0.975343,0.950602,19:09
7,0.064181,0.084857,0.974961,0.950703,19:09
8,0.061764,0.082080,0.976198,0.953352,19:13
9,0.057575,0.082465,0.976158,0.953059,19:08


In [10]:
learn.dbunch = get_data(256, 384, splits=Meta.splits_stg1, img_dir=path_jpg)
do_fit(learn, 4, 4e-4)
learn.save(f'runs/{name}-3')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.065102,0.081450,0.975506,0.952422,46:03
1,0.060117,0.079891,0.976014,0.953326,45:27
2,0.057547,0.079263,0.976101,0.953377,45:17
3,0.052879,0.080095,0.976213,0.953479,45:19


## Submission

In [8]:
learn.load(f'runs/{name}-3')

In [12]:
# tst_fns = df_tst.index.values

In [9]:
sub_fn = f'subm/{name}-3'

In [10]:
learn.dbunch = get_test_data(Meta.df_tst, bs=512, sz=384)

In [15]:
# tst_bs = 256
# tst_sz = 384
# tst_splits = [L.range(tst_fns), L.range(tst_fns)]
# tst_dbch = get_data_gen(tst_fns, bs=tst_bs, img_tfm=get_pil_fn(path/'tst_jpg'), sz=tst_sz, splits=tst_splits, test=True)
# learn.dbunch = tst_dbch

In [11]:
preds,targs = learn.get_preds()

In [12]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [13]:
FileLink(f'{sub_fn}.csv')

/home/ashaw/kaggle/rsna_retro/subm/baseline_full_stg1_xse_resnext-3.csv

In [14]:
api.competition_submit(f'{sub_fn}.csv', 'Training on Stage 1 (384), Valid on Stage 2. - 0.084161', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 11.2MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [22]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14266028,
 'totalBytes': 27277209,
 'date': '2020-01-22T18:34:09.763Z',
 'description': 'Training on Stage 1 (384), Valid on Stage 2. - 0.084161',
 'errorDescription': None,
 'fileName': 'baseline_full_stg1_xse_resnext-3.csv',
 'publicScore': '1.04722',
 'privateScore': '0.06479',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14266028/14266028.raw'}